**Общая информация**

**Срок сдачи:** 21 октября 2019, 08:30 

**Штраф за опоздание:** по 0.5 балла за 24 часа задержки. Через 10 дней домашнее задание сгорает.

При отправлении ДЗ указывайте фамилию в названии файла
Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:

[ML0919, Задание 1] Фамилия Имя.

Используйте данный Ipython Notebook при оформлении домашнего задания.


**Штрафные баллы:**

1. Невыполнение PEP8 -0.5 баллов
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw1.ipynb) -0.5 баллов
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -0.5 баллов


In [1]:
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn import datasets
from sklearn.base import ClassifierMixin
from sklearn.datasets import fetch_mldata, fetch_20newsgroups

from sklearn.neighbors.base import NeighborsBase, KNeighborsMixin, SupervisedIntegerMixin 
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KDTree

from scipy.spatial.distance import cdist
%load_ext pycodestyle_magic

##### Задание 1 (1 балл)
Реализовать KNN в классе MyKNeighborsClassifier (обязательное условие: точность не ниже sklearn реализации)
Разберитесь самостоятельно, какая мера расстояния используется в KNeighborsClassifier дефолтно и реализуйте свой алгоритм именно с этой мерой. Самостоятельно разберитесь, как считается score из KNeighborsClassifier и реализуйте аналог в своём классе. Score не должен уступать значению KNN из sklearn

In [2]:
%%pycodestyle


class MyKNeighborsClassifier(NeighborsBase,
                             KNeighborsMixin,
                             SupervisedIntegerMixin,
                             ClassifierMixin):

    def __init__(self, n_neighbors, algorithm='brute'):
        self.n_neighbors = n_neighbors
        self.algorithm = algorithm

    def fit(self, X, y):
        self.points = np.array(X)
        self.points_class = np.array(y)
        self.classes = len(set(y))
        if self.algorithm == 'kd_tree':
            self.tree = KDTree(X)

    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)

    def find_kneighbours(self, x):
        return np.argpartition(cdist(x[np.newaxis, :],
                                     self.points,
                                     'minkowski'),
                               self.n_neighbors)[:self.n_neighbors]

    def predict_proba(self, X):
        p = []
        if self.algorithm == 'brute':
            for x in X:
                indexes = self.find_kneighbours(x)
                prob = np.zeros(self.classes)
                for i in range(self.n_neighbors):
                    prob[self.points_class[indexes[0, i]]] += 1
                p.append(prob)
    #            print(prob)
        elif self.algorithm == 'kd_tree':
            indexes = self.tree.query(X, k=self.n_neighbors,
                                      return_distance=False)
            for index in indexes:
                prob = np.zeros(self.classes)
                for i in index:
                    prob[self.points_class[i]] += 1
                p.append(prob)
        return p

    def score(self, X, y):
        p = self.predict(X)
        return len(np.where(p == y)[0]) / len(y)


**IRIS**

В библиотеке scikit-learn есть несколько датасетов из коробки. Один из них [Ирисы Фишера](https://ru.wikipedia.org/wiki/%D0%98%D1%80%D0%B8%D1%81%D1%8B_%D0%A4%D0%B8%D1%88%D0%B5%D1%80%D0%B0)

In [3]:
iris = datasets.load_iris()

In [4]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [5]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='brute')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='brute')

In [6]:
clf.fit(X_train, y_train)
my_clf.fit(X_train, y_train)

In [7]:
assert abs(my_clf.score(X_test, y_test) - clf.score(X_test,y_test))<0.005, "Score must be simillar"

**Задание 2 (0.5 балла)**

Давайте попробуем добиться скорости работы на fit, predict и predict_proba сравнимой со sklearn для iris.
Для этого используем numpy

In [8]:
%time clf.fit(X_train, y_train)

Wall time: 1e+03 µs


KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [9]:
%time my_clf.fit(X_train, y_train)

Wall time: 0 ns


In [10]:
%time clf.predict(X_test)

Wall time: 2 ms


array([2, 2, 0, 2, 2, 1, 0, 2, 1, 0, 1, 0, 1, 0, 1])

In [11]:
%time my_clf.predict(X_test)

Wall time: 3 ms


array([2, 2, 0, 2, 2, 1, 0, 2, 1, 0, 1, 0, 1, 0, 1], dtype=int64)

In [12]:
%time clf.predict_proba(X_test)

Wall time: 2 ms


array([[0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [13]:
%time my_clf.predict_proba(X_test)

Wall time: 3 ms


[array([0., 0., 2.]),
 array([0., 0., 2.]),
 array([2., 0., 0.]),
 array([0., 0., 2.]),
 array([0., 0., 2.]),
 array([0., 2., 0.]),
 array([2., 0., 0.]),
 array([0., 0., 2.]),
 array([0., 2., 0.]),
 array([2., 0., 0.]),
 array([0., 2., 0.]),
 array([2., 0., 0.]),
 array([0., 2., 0.]),
 array([2., 0., 0.]),
 array([0., 2., 0.])]

###### Задание 3 (1 балл)
Добавьте algorithm='kd_tree' в реализацию KNN (использовать KDTree из sklearn.neighbors). Необходимо добиться скорости работы на fit,  predict и predict_proba сравнимой со sklearn для iris.
Для этого используем numpy. Score не должен уступать значению KNN из sklearn

In [14]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')

In [15]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [16]:
%time clf.fit(X_train, y_train)

Wall time: 999 µs


KNeighborsClassifier(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [17]:
%time my_clf.fit(X_train, y_train)

Wall time: 0 ns


In [18]:
print(clf.score(X_test,y_test))
print(my_clf.score(X_test, y_test))

0.9333333333333333
0.9333333333333333


In [19]:
%time clf.predict(X_test)

Wall time: 2 ms


array([0, 1, 1, 2, 2, 2, 1, 0, 0, 1, 2, 1, 1, 0, 0])

In [20]:
%time my_clf.predict(X_test)

Wall time: 0 ns


array([0, 1, 1, 2, 2, 2, 1, 0, 0, 1, 2, 1, 1, 0, 0], dtype=int64)

In [21]:
%time clf.predict_proba(X_test)

Wall time: 2 ms


array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [22]:
%time my_clf.predict_proba(X_test)

Wall time: 0 ns


[array([2., 0., 0.]),
 array([0., 2., 0.]),
 array([0., 2., 0.]),
 array([0., 0., 2.]),
 array([0., 0., 2.]),
 array([0., 0., 2.]),
 array([0., 2., 0.]),
 array([2., 0., 0.]),
 array([2., 0., 0.]),
 array([0., 2., 0.]),
 array([0., 0., 2.]),
 array([0., 2., 0.]),
 array([0., 2., 0.]),
 array([2., 0., 0.]),
 array([2., 0., 0.])]

In [23]:
assert abs(my_clf.score(X_test, y_test) - clf.score(X_test,y_test))<0.005, "Score must be simillar"

**Задание 4 (2.5 балла)**

Рассмотрим новый датасет 20 newsgroups

In [24]:
newsgroups = fetch_20newsgroups(subset='train',remove=['headers','footers', 'quotes'])

In [25]:
data = newsgroups['data']
target = newsgroups['target']

In [26]:
print(len(target))
print(len(data))

11314
11314


In [27]:
target

array([7, 4, 4, ..., 3, 1, 8])

Переведите во всех документах все буквы в нижний регистр и замените во всех документах символы, не
являющиеся буквами и цифрами, на пробелы. Далее разбейте текста по пробельным символам на токены(термы/слова). Удалите текста, содержащие только пробелы.

In [28]:
def clean_text(text):
    return "".join(c 
                   if c.isdigit() or c.isalpha()
                   else " " 
                   for string in text for c in string)

def check_text(text):
    return np.any([True if bool(string or not string.isspace()) else False for string in text])

In [29]:
cleaned_data = [clean_text(text) for text in [text.lower() for text in data]]
cleaned_data[0]

'i was wondering if anyone out there could enlighten me on this car i saw the other day  it was a 2 door sports car  looked to be from the late 60s  early 70s  it was called a bricklin  the doors were really small  in addition  the front bumper was separate from the rest of the body  this is  all i know  if anyone can tellme a model name  engine specs  years of production  where this car is made  history  or whatever info you have on this funky looking car  please e mail '

In [30]:
parsed_data = [text.split() for text in cleaned_data]

In [31]:
indeces = []
for i in range(len(parsed_data)):
    if check_text(parsed_data[i]):
        indeces.append(i)

In [32]:
data_tok = [parsed_data[i] for i in indeces]
# data_tok should be a list of lists of tokens for each line in data.

In [33]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower() , map(' '.join, data_tok))), "please make sure that you lowercase the data and drop spaced texts"

Преобразуйте датасет в разреженную матрицу scipy.sparse.csr_matrix, где значение x в позиции (i, j)
означает, что в документе i слово j встретилось x раз

In [34]:
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split

In [35]:
len(data_tok)

11004

In [36]:
indptr = [0]
indices = []
data = []
vocabulary = {}
for d in data_tok:
    for term in d:
        index = vocabulary.setdefault(term, len(vocabulary))
        indices.append(index)
        data.append(1)
    indptr.append(len(indices))

In [37]:
len(indices)

2408316

In [38]:
X = csr_matrix((data, indices, indptr), dtype=int).toarray()
y = [target[i] for i in indeces]

In [39]:
print(len(X))
print(len(X[0]))

11004
98749


In [40]:
count_words = np.sum(X, axis = 0)

In [41]:
len(count_words[count_words > 25]), len(count_words[count_words < 1000])

(7227, 98505)

In [42]:
max(count_words), sum(count_words)

(106478, 2408316)

In [43]:
def delete_sparse_words(X, n=100, N=1000):
    index = []
    X = np.array(X)
    return X[:, np.logical_and(count_words > n, count_words < N)]

In [44]:
X_new = delete_sparse_words(X)

In [45]:
X_new.shape

(11004, 2183)

*Так мы получили векторное представление наших текстов. Значит можно приступать к задаче обучения модели*

Реализуйте разбиение выборки для кросс-валидации на 3 фолдах. Разрешено использовать sklearn.cross_validation

In [48]:
from sklearn.model_selection import cross_validate

In [49]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='brute')

In [50]:
cv_results = cross_validate(clf, X_new, y, cv=3)

In [51]:
cv_results

{'fit_time': array([0.77554631, 0.78755641, 0.67047405]),
 'score_time': array([2.68190765, 1.79826975, 1.97240591]),
 'test_score': array([0.24748436, 0.23636859, 0.23776988])}

Напишите метод, позволяющий найти оптимальное количество ближайших соседей(дающее максимальный score в среднем на валидации на 3 фолдах).
Постройте график зависимости среднего score от количества соседей. Можно рассмотреть число соседей от 1 до 10.

In [52]:
cv_results = []
for k in range(1, 11):
    clf = KNeighborsClassifier(n_neighbors=k, algorithm='brute')
    cv_results.append(cross_validate(clf, X_new, y, cv=3))

In [53]:
result = []
for k in range(10):
    result.append(cv_results[k]["test_score"].mean())
max(result), np.argmax(result)+1

(0.2872520842871686, 1)

Как изменится качество на валидации, если:

1. Используется косинусная метрика вместо евклидовой.
2. К текстам применяется TfIdf преобразование( sklearn.feature_extraction.text.TfidfTransformer)

Сравните модели, выберите лучшую.

In [54]:
import sklearn.neighbors
from sklearn.feature_extraction.text import TfidfTransformer

In [55]:
TfidfTransformer(X_new)

TfidfTransformer(norm=array([[1, 4, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]),
                 smooth_idf=True, sublinear_tf=False, use_idf=True)

In [56]:
cv_results2 = []
for k in range(1, 11):
    clf = KNeighborsClassifier(n_neighbors=k, algorithm='brute', metric='cosine')
    cv_results2.append(cross_validate(clf, X_new, y, cv=3))

In [57]:
result2 = []
for k in range(10):
    result2.append(cv_results2[k]["test_score"].mean())
max(result2), np.argmax(result2)+1

(0.511625036171885, 10)

Загрузим  теперь test  часть нашей выборки и преобразуем её аналогично с train частью. Не забудьте, что наборы слов в train и test части могут отличаться.

In [ ]:
newsgroups = fetch_20newsgroups(subset='test',remove=['headers','footers', 'quotes'])

Оценим качество(score) вашей лучшей модели на test части датасета. Отличается ли оно от кросс-валидации? Попробуйте сделать выводы, почему отличается качество.